In [1]:
import pandas as pd
import ast
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools

In [2]:
def get_data(merchant_name):
    df_full = pd.read_csv("synthetic_transaction_data_Dining_SMALL_w_items.csv")
    df_full['items'] = [", ".join(ast.literal_eval(x)) for x in list(df_full['items'])]
    df = df_full[df_full['merchant_name'] == merchant_name].copy()
    return df 

In [3]:
df_md = get_data("McDonald's")
df_md.head()

,transaction_id,transaction_date,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag,cardholder_name,items,prices
16728,PFJNOT-JDOQ-W4LBOV,2023-06-21 09:36,4039748374251628,visa,McDonald's,Fast Food,New Mexico,Rio Rancho,Pending,3.99,MCC 5814,0,Christy Farrell,Big Mac,[3.99]
16729,1MPEOS-2BRC-7Y7HHW,2023-03-02 15:57,4374333274609192,visa,McDonald's,Fast Food,Nebraska,Omaha,Declined,3.99,MCC 5814,1,Jennifer Pearson,Big Mac,[3.99]
16730,B3KGV8-XX5L-L5F5FT,2023-03-17 18:06,4057753928404951,visa,McDonald's,Fast Food,Tennessee,Knoxville,Pending,22.33,MCC 5814,0,Richard Fuller,"Big Mac, Quarter Pounder with Cheese, Chicken ...","[3.99, 4.79, 4.49, 3.79, 1.29, 1.59, 2.39]"
16731,UBOL24-8AP4-2ULPG5,2023-02-07 19:24,4053151982662644,visa,McDonald's,Fast Food,Delaware,Dover,Successful,3.99,MCC 5814,0,Lisa Stewart,Big Mac,[3.99]
16732,7X6HV4-TCHL-DT0ZCX,2023-01-15 07:05,4184379659883563,visa,McDonald's,Fast Food,Oregon,Salem,Declined,3.99,MCC 5814,0,Andrea Faulkner,Big Mac,[3.99]


In [4]:
df_sw = get_data('Subway')
df_sw.head()

,transaction_id,transaction_date,card_number,card_type,merchant_name,merchant_category,merchant_state,merchant_city,transaction_status,transaction_amount,merchant_category_code,fraud_flag,cardholder_name,items,prices
6608,1ANBFI-TOFZ-J1FHXR,2023-02-17 07:16,4318249446099857,visa,Subway,Fast Food,New Hampshire,Manchester,Declined,6.59,MCC 5814,0,James Lee,Italian BMT,[6.59]
6609,UIF4NV-FC9E-XKCNBG,2023-03-22 17:22,4101211350337682,visa,Subway,Fast Food,New Hampshire,Nashua,Declined,6.59,MCC 5814,0,Gabriella Hicks,Italian BMT,[6.59]
6610,75UXRX-LGYI-JC802K,2023-06-14 16:49,4436281502955562,visa,Subway,Fast Food,Maine,Lewiston,Successful,6.59,MCC 5814,0,Mr. Eduardo Ford,Italian BMT,[6.59]
6611,3D1LES-NSWC-UGGOMS,2023-03-23 07:13,4426321457580551,visa,Subway,Fast Food,Georgia,Augusta,Declined,13.08,MCC 5814,1,Justin Francis,"Italian BMT, Oven Roasted Chicken","[6.59, 6.49]"
6612,8G8LEE-313G-XFTWHT,2023-03-13 22:42,4947626214893547,visa,Subway,Fast Food,New Hampshire,Nashua,Pending,6.59,MCC 5814,0,Sara Reese,Italian BMT,[6.59]


In [5]:
print(df_sw.columns)

Index(['transaction_id', 'transaction_date', 'card_number', 'card_type',
       'merchant_name', 'merchant_category', 'merchant_state', 'merchant_city',
       'transaction_status', 'transaction_amount', 'merchant_category_code',
       'fraud_flag', 'cardholder_name', 'items', 'prices'],
      dtype='object')


In [6]:
df = df_md
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
NOW = df['transaction_date'].max()
rfmTable = df.groupby('cardholder_name').agg({'transaction_date': lambda x: (NOW - x.max()).days, 'transaction_id': lambda x: len(x), 'transaction_amount': lambda x: x.sum()})
rfmTable['transaction_date'] = rfmTable['transaction_date'].astype(int)
rfmTable.rename(columns={'transaction_date': 'recency', 
                         'transaction_id': 'frequency',
                         'transaction_amount': 'monetary_value'}, inplace=True)
rfmTable = rfmTable.reset_index()

In [7]:
rfmTable['r_quartile'] = pd.qcut(rfmTable['recency'], q=4, labels=range(1,5), duplicates='raise')
rfmTable['f_quartile'] = pd.qcut(rfmTable['frequency'], q=4, labels=range(1,5), duplicates='drop')
rfmTable['m_quartile'] = pd.qcut(rfmTable['monetary_value'], q=4, labels=range(1,5), duplicates='drop')
rfm_data = rfmTable.reset_index()

In [8]:
rfm_data['r_quartile'] = rfm_data['r_quartile'].astype(str)
rfm_data['f_quartile'] = rfm_data['f_quartile'].astype(str)
rfm_data['m_quartile'] = rfm_data['m_quartile'].astype(str)
rfm_data['RFM_score'] = rfm_data['r_quartile'] + rfm_data['f_quartile'] + rfm_data['m_quartile']
rfm_data = rfm_data.reset_index()

In [9]:
rfm_data['customer_segment'] = 'Other'

rfm_data.loc[rfm_data['RFM_score'].isin(['334', '443', '444', '344', '434', '433', '343', '333']), 'customer_segment'] = 'Premium Customer' #nothing <= 2
rfm_data.loc[rfm_data['RFM_score'].isin(['244', '234', '232', '332', '143', '233', '243']), 'customer_segment'] = 'Repeat Customer' # f >= 3 & r or m >=3
rfm_data.loc[rfm_data['RFM_score'].isin(['424', '414', '144', '314', '324', '124', '224', '423', '413', '133', '323', '313', '134']), 'customer_segment'] = 'Top Spender' # m >= 3 & f or m >=3
rfm_data.loc[rfm_data['RFM_score'].isin([ '422', '223', '212', '122', '222', '132', '322', '312', '412', '123', '214']), 'customer_segment'] = 'At Risk Customer' # two or more  <=2
rfm_data.loc[rfm_data['RFM_score'].isin(['411','111', '113', '114', '112', '211', '311']), 'customer_segment'] = 'Inactive Customer' # two or more  =1

del rfm_data['index']

In [10]:
rfm_data.head()

,level_0,cardholder_name,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFM_score,customer_segment
0,0,Aaron Nelson,46,4,47.56,3,2,3,323,Top Spender
1,1,Alicia Ballard,43,3,16.76,3,1,1,311,Inactive Customer
2,2,Alison Watson,23,4,15.96,2,2,1,221,Other
3,3,Alison White,41,9,45.49,3,4,3,343,Premium Customer
4,4,Alyssa Phelps,1156,1,3.99,4,1,1,411,Inactive Customer


In [11]:
print(Counter(rfm_data['customer_segment']))

Counter({'Inactive Customer': 68, 'At Risk Customer': 67, 'Top Spender': 57, 'Repeat Customer': 48, 'Premium Customer': 43, 'Other': 21})


In [12]:
def generate_recommendations(target_customer, cohort, num_recommendations=10):
    user_item_matrix = cohort.groupby('cardholder_name')['items'].apply(lambda x: list(set([', '.join(x)]))).reset_index()
    user_item_matrix['items'] = [', '.join(list(set(list(x[0].split(", "))))) for x in list(user_item_matrix['items'])]
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(user_item_matrix['items'])

    similarity_matrix = cosine_similarity(tfidf_matrix)    
    
    target_customer_index = user_item_matrix[user_item_matrix['cardholder_name'] == target_customer].index[0]
    similar_customers = similarity_matrix[target_customer_index].argsort()[::-1][1:num_recommendations+1]

    target_customer_purchases = set(user_item_matrix[user_item_matrix['cardholder_name'] == target_customer]['items'].iloc[0])
    
    recommendations = []
    
    for customer_index in similar_customers:
        customer_purchases = set([user_item_matrix.iloc[customer_index]['items']])
        new_items = customer_purchases.difference(target_customer_purchases)
        recommendations.extend(new_items)

    recommendations = [item.split(', ') for item in recommendations]
    recommendations = list(itertools.chain.from_iterable(recommendations))
    return recommendations


In [13]:
def customer_analysis(name, cohort):    
    recommendations = generate_recommendations(name, cohort, num_recommendations=10)
    df = cohort[cohort['cardholder_name'] == name]
    already = list(df['items'])
    already = [item.split(', ') for item in already]
    already = list(itertools.chain.from_iterable(already))
    recs = set(recommendations) - set(already)
    return recs

In [14]:
rfm_data = rfm_data[rfm_data['customer_segment']== 'Premium Customer']
premium = list(set(rfm_data['cardholder_name']))
df_premium = df[df['cardholder_name'].isin(premium)]

In [15]:
names = dict()
for name in list(df_premium['cardholder_name']):
    recs = customer_analysis(name, df_premium)
    if recs and len(recs) >= 2 and name not in names:
        names[name] = recs
        print(f"Recommendations for {name}: ", recs )

Recommendations for Sara Rich:  {'French Fries (Medium)', 'Apple Pie'}
Recommendations for Heather Foster:  {'McFlurry', 'Cheeseburger'}
Recommendations for Renee Stephenson:  {'McFlurry', 'McDouble'}


In [17]:
names = dict()
for name in list(df['cardholder_name']):
    recs = customer_analysis(name, df)
    if recs and len(recs) > 0 and name not in names:
        names[name] = recs
        print(f"Recommendations for {name}: ", recs )

Recommendations for Megan Jones:  {'McDouble'}
Recommendations for Jennifer Downs:  {'Apple Pie'}
Recommendations for Yolanda Wong:  {'Apple Pie'}
Recommendations for Stephanie Medina:  {'Apple Pie'}
Recommendations for Jennifer Robbins:  {'McDouble'}
Recommendations for Deborah Houston:  {'McDouble'}
Recommendations for Patrick Nichols:  {'McDouble'}
Recommendations for Kimberly Davies:  {'Apple Pie'}
Recommendations for Jessica Russell:  {'McFlurry'}
Recommendations for William Baldwin:  {'McFlurry'}
Recommendations for Megan Ramirez:  {'Apple Pie'}
Recommendations for Christopher Wilson:  {'Apple Pie'}
Recommendations for Richard Tucker:  {'McDouble'}
Recommendations for Matthew Reed:  {'McDouble'}
Recommendations for Mark Ramirez:  {'McDouble'}
Recommendations for Timothy Willis:  {'McDouble'}
Recommendations for Jonathan Jones MD:  {'Apple Pie'}
Recommendations for Heather Foster:  {'McFlurry'}
Recommendations for Johnny Walker:  {'Apple Pie'}
Recommendations for Renee Stephenson:

In [18]:
df = df_sw
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
NOW = df['transaction_date'].max()
rfmTable = df.groupby('cardholder_name').agg({'transaction_date': lambda x: (NOW - x.max()).days, 'transaction_id': lambda x: len(x), 'transaction_amount': lambda x: x.sum()})
rfmTable['transaction_date'] = rfmTable['transaction_date'].astype(int)
rfmTable.rename(columns={'transaction_date': 'recency', 
                         'transaction_id': 'frequency',
                         'transaction_amount': 'monetary_value'}, inplace=True)
rfmTable = rfmTable.reset_index()

In [19]:
rfmTable['r_quartile'] = pd.qcut(rfmTable['recency'], q=4, labels=range(1,5), duplicates='raise')
rfmTable['f_quartile'] = pd.qcut(rfmTable['frequency'], q=4, labels=range(1,5), duplicates='drop')
rfmTable['m_quartile'] = pd.qcut(rfmTable['monetary_value'], q=4, labels=range(1,5), duplicates='drop')
rfm_data = rfmTable.reset_index()

In [20]:
rfm_data['r_quartile'] = rfm_data['r_quartile'].astype(str)
rfm_data['f_quartile'] = rfm_data['f_quartile'].astype(str)
rfm_data['m_quartile'] = rfm_data['m_quartile'].astype(str)
rfm_data['RFM_score'] = rfm_data['r_quartile'] + rfm_data['f_quartile'] + rfm_data['m_quartile']
rfm_data = rfm_data.reset_index()

In [21]:
rfm_data['customer_segment'] = 'Other'

rfm_data.loc[rfm_data['RFM_score'].isin(['334', '443', '444', '344', '434', '433', '343', '333']), 'customer_segment'] = 'Premium Customer' #nothing <= 2
rfm_data.loc[rfm_data['RFM_score'].isin(['244', '234', '232', '332', '143', '233', '243']), 'customer_segment'] = 'Repeat Customer' # f >= 3 & r or m >=3
rfm_data.loc[rfm_data['RFM_score'].isin(['424', '414', '144', '314', '324', '124', '224', '423', '413', '133', '323', '313', '134']), 'customer_segment'] = 'Top Spender' # m >= 3 & f or m >=3
rfm_data.loc[rfm_data['RFM_score'].isin([ '422', '223', '212', '122', '222', '132', '322', '312', '412', '123', '214']), 'customer_segment'] = 'At Risk Customer' # two or more  <=2
rfm_data.loc[rfm_data['RFM_score'].isin(['411','111', '113', '114', '112', '211', '311']), 'customer_segment'] = 'Inactive Customer' # two or more  =1

del rfm_data['index']

In [22]:
rfm_data = rfm_data[rfm_data['customer_segment']== 'Premium Customer']
premium = list(set(rfm_data['cardholder_name']))
df_premium = df[df['cardholder_name'].isin(premium)]

In [23]:
names = dict()
for name in list(df_premium['cardholder_name']):
    recs = customer_analysis(name, df_premium)
    if recs and len(recs) >= 2 and name not in names:
        names[name] = recs
        print(f"Recommendations for {name}: ", recs )

Recommendations for Jessica White:  {'Cold Cut Combo', 'Roast Beef', 'Chicken Teriyaki'}
Recommendations for Jennifer Lee:  {'Roast Beef', 'Chicken Teriyaki'}
Recommendations for Travis Jackson:  {'Tuna', 'Veggie Delite'}
Recommendations for Sara Sosa:  {'Tuna', 'Veggie Delite'}


In [24]:
names = dict()
customers = list(set(df['cardholder_name']))
for name in customers:
    recs = customer_analysis(name, df)
    if recs and len(recs) >= 1 and name not in names:
        names[name] = recs
        print(f"Recommendations for {name}: ", recs )

Recommendations for Jennifer Lee:  {'Roast Beef', 'Chicken Teriyaki'}
Recommendations for Jessica White:  {'Cold Cut Combo'}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c4b20a38-75f0-406e-873e-9c308a030295' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>